# Data, Machines and the 🐍 
<img src="https://raw.githubusercontent.com/habermanUIUC/CodeStoryLessons/main/lessons/dmap/data/normalization/html/section00.png" align="left"/>

<a id="install"></a>
## Notebook Preparation for Lesson 1•2•3
Each lesson will start with a similar template (given in the course schedule):  
1. **save** to your google drive (copy to drive)<br/><img src="https://raw.githubusercontent.com/habermanUIUC/CodeStoryLessons/main/assets/images/colab/copy-to-drive.png"/>
2. **update** the NET_ID to be your netID (no need to include @illinois.edu)
3. **run** the next cell to install the IDE. <img src="https://raw.githubusercontent.com/habermanUIUC/CodeStoryLessons/main/assets/images/colab/play-button.png"/>

In [0]:
LESSON_ID = 'dmap:data:normalization'   # keep this as is
NET_ID    = 'CHANGE_ME' # CHANGE_ME to your netID (keep the quotes)

def install_ide(net_id, lesson_id):
  import sys
  if 'codestories' not in sys.modules:
      print('installing modules')
      !pip install git+https://mehaberman@bitbucket.org/mehaberman/codestories.git --upgrade &> install.log
  
  from codestories.cs.CodeStories import CodeStory
  return CodeStory(net_id, lesson_id)

ide = install_ide(NET_ID, LESSON_ID)
print(ide.welcome())

# Lesson Data Normalization (part 1)
(hit ▶ to read the first part of the lesson️)

In [0]:
# run to read the next section
ide.reader.view_section(1)

# Cleaning the Mssing

In [0]:
# run to read the next section
ide.reader.view_section(2)

In [0]:
import LessonUtil as Util
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

def build_titanic():
  df = pd.read_csv(Util.path_for_data('titanic.csv'))
  print('total rows', len(df))
  
  # add an extra passenger
  extra = {'name': 'Jack Dawson', 'age': 28, 'id': len(df)+1, 'gender': 'male'}
  df = df.append(extra, ignore_index=True)
  
  # add an extra field for using a custom transformer
  df['sid'] = df['age'].apply(lambda x: 'NA' if np.isnan(x) else "{:.0f}".format(1912-x)).astype('string')
  df['sid'].replace('NA', np.nan, inplace=True)
  
  return df.copy()

df = build_titanic()
print(df.head(1))

In [0]:
# run to read the next section
ide.reader.view_section(4)

In [0]:
def show_missing(df):
  print(df.isna().sum())

show_missing(df)

In [0]:
# run to read the next section
ide.reader.view_section(6)

In [0]:
def process_missing_age(df, debug=True):
 
  # mask to select the rows where age is empty
  mask = df.age.isna()

  # calculate the mean (the replacement value)
  replace_value = df.age.mean()
  
  # fill those values with the value calculated
  df['age_clean']= df[mask].age.fillna(replace_value)
  if debug:
    # print out the updates
    cols = ['id', 'name', 'age', 'age_clean']
    print(df[mask][cols].head())
    
# pass in a copy, so we keep the original
process_missing_age(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(8)

In [0]:
def process_missing_age_skl(df, debug=True):
  from sklearn.impute import SimpleImputer
  import numpy as np

  # np.nan is the how pandas marks missing values
  # replace with the mean
  imr = SimpleImputer(strategy="mean", missing_values=np.nan)

  # Impute values
  values = df.age.values.reshape(-1,1)  
  out = imr.fit_transform(values)
  
  # now we assign those values to a new column
  df['im_age'] = out

  if debug:
    cols = ['id', 'name', 'age', 'im_age']
    mask = df.age.isna()
    print(df[mask][cols].head())
    print('Missing')
    print(df.isna()[cols].sum())

process_missing_age_skl(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(10)

# Categorical Data

In [0]:
# run to read the next section
ide.reader.view_section(11)

In [0]:
def category_to_number(df, col_name, new_name, debug=True):

  # map the categories to unique integers (starting at 0)
  df[new_name] = df[col_name].astype("category").cat.codes

  if debug:
    values = df[col_name].unique()
    print('{:d} unique values for {:s}:'.format(len(values), col_name), values)
    # show how many rows are in each group
    print(df.groupby([col_name]).count()[new_name].reset_index())
    print(df.groupby([col_name]).count()[new_name].sum())
    
  return df
  
# map gender to a 0/1 code
cols = ['gender', 'g_code']
df2 = category_to_number(df.copy(), *cols)  # putting * to good use
print(df2[cols][0:10]) # first 10 rows

In [0]:
# run to read the next section
ide.reader.view_section(13)

In [0]:
# map port of embarkation (C, Q, S) 
# C = Cherbourg, Q = Queenstown, S = Southampton,  B = ??
cols = ['embarked', 'e_code']
df = category_to_number(df.copy(), *cols)  
print(df[cols][0:10]) # first 10 rows
mask = df.embarked.isna()
print(df[mask])

In [0]:
# run to read the next section
ide.reader.view_section(15)

In [0]:
def map_class_attribute(df, debug=True):

  # map 1st/2nd/3rd class as ordinal 1st < 2nd < 3rd + missing
  # replace nan with the value 'unknown'
  df['class'].fillna('unknown', inplace=True)
  # assign the labels values
  ord_map = {'3rd':3, '2nd':2, '1st':1, 
             'engineering crew':4, 
             'victualling crew':4, 
             'restaurant staff':4, 
             'deck crew':4, 'unknown':0}
            
  # apply the map to the 'class' attribute
  df['o_class'] = df['class'].map(ord_map)
 
  if debug:
    print(df['class'].unique().tolist())
    # show how many rows are in each group
    print(df.groupby(['class']).count()['o_class'].reset_index())

  return df

df_class = map_class_attribute(df.copy())
print(df_class.head())

In [0]:
# run to read the next section
ide.reader.view_section(17)

In [0]:
def print_passenger_class_stats(df):
  is_crew = df['o_class'].isin([4])

  # any of these will work
  is_pass = df['o_class'].isin([1,2,3])
  is_pass = (df['o_class'] < 4 ) & (df['o_class'] > 0)
  is_pass = ~is_crew  # will include the unknowns
  
  print('crew', len(df[is_crew]))
  print('pass', len(df[is_pass]))

# this assumes map_class_attribute is done
print_passenger_class_stats(df_class.copy())

In [0]:
# run to read the next section
ide.reader.view_section(19)

In [0]:
def create_binary_field(df, debug=True):
  # Binary Fields
  from sklearn.preprocessing import Binarizer
  binarizer = Binarizer(threshold=3, copy=True)  # <= 3 asssing 
  column_values = binarizer.fit_transform(df.o_class.values.reshape(-1, 1))
  
  # flip the values (1 -> 0; 0 -> 1)
  df['is_passenger'] = 1 - column_values  
  if debug:
    print(column_values)
    print(df.head(5))

print(create_binary_field(df_class.copy()))

In [0]:
# run to read the next section
ide.reader.view_section(21)

In [0]:
def one_hot_encoding(df):
  import numpy as np
  from sklearn.preprocessing import OneHotEncoder
  
  onehot = OneHotEncoder(dtype=np.int, sparse=True)

  # fill in any missing values with 'UNK'
  df['embarked'].fillna('UKN', inplace=True)
  values = df['embarked'].values.reshape(-1, 1)
  values = onehot.fit_transform(values).toarray() # it is sparse
  labels = onehot.categories_

  return pd.DataFrame(values, columns=labels)
  
df_hot = one_hot_encoding(df.copy())
print(df_hot.tail(20))

In [0]:
# run to read the next section
ide.reader.view_section(23)

In [0]:
# binning data
def bin_demo1(df):
  # set up custom bins
  bins =   [ 0,       3,      8,      16,           21,     35,      55,  200] 
  labels = ['infant','child','youth','young adult','adult','middle','senior']
  age_bins = pd.cut(df['age'], bins=bins, labels=labels, right=False)
  df['age_cat'] = age_bins
  print(df['id age age_cat'.split()].head(10))
  return df

df_bin = bin_demo1(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(25)

In [0]:
def bin_demo2(df):
  # uniform bins
  from sklearn.preprocessing import KBinsDiscretizer

  # this must be done first
  df['age'].fillna(df['age'].mean(), inplace=True)
  
  binner = KBinsDiscretizer(n_bins=8, encode='ordinal', strategy='uniform')
  values = binner.fit_transform(df['age'].values.reshape(-1, 1))
  df['age_cat2'] = values
  print(df['id age age_cat age_cat2'.split()].head(10))

bin_demo2(df_bin.copy())

In [0]:
# run to read the next section
ide.reader.view_section(27)

In [0]:
def bin_demo3(df):
  # one hot binning
  from sklearn.preprocessing import KBinsDiscretizer

  bin_count = 4
  df['age'].fillna(df['age'].mean(), inplace=True)
  binner = KBinsDiscretizer(n_bins=bin_count, encode='onehot-dense', strategy='uniform')
  values = binner.fit_transform(df['age'].values.reshape(-1, 1))
  labels = ['bin {:d}'.format(i) for i in range(1, bin_count+1)]

  df2 = pd.DataFrame(values, columns=labels)
  df2['age'] = df['age']
  print(df2.head(10))

bin_demo3(df.copy())

# Numeric Data

In [0]:
# run to read the next section
ide.reader.view_section(29)

# Feature Scaling Data

In [0]:
# run to read the next section
ide.reader.view_section(30)

In [0]:
import numpy as np

def fare_stats(df):
  df['fare'].fillna(df['fare'].mean(), inplace=True)

  print(df['fare'].min())
  min_fair = np.min(df['fare'])
  max_fair = np.max(df['fare'])
  print(min_fair, max_fair)
  print(np.ptp(df['fare']))

df = build_titanic()
fare_stats(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(32)

In [0]:
def fare_min_max_scaled_np(df):
  pass

In [0]:
# run to read the next section
ide.reader.view_section(34)

In [0]:
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler

def fare_min_max_scaled(df):
  df['fare_scaled'] = minmax_scale(df['fare'])

  # fit & transform way
  scaler = MinMaxScaler()
  s_values = scaler.fit_transform(df['fare'].values.reshape(-1,1))
  df['fare_scaled2'] = s_values
  print(df['fare fare_scaled fare_scaled2'.split()].head(10))

fare_min_max_scaled(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(36)

In [0]:
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

def fare_z_scaled(df):

  # simple way
  df['fare_z'] = scale(df['fare'])

  # fit & transform way
  scaler = StandardScaler()
  s_values = scaler.fit_transform(df['fare'].values.reshape(-1,1))
  df['fare_z2'] = s_values
  print(df['fare fare_z fare_z2'.split()].head(10))

fare_z_scaled(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(38)

In [0]:
import numpy as np
def find_fare_outliers(df):
  df['fare'].fillna(df['fare'].mean(), inplace=True)
  data = df['fare'].values.reshape(-1,1)

  m = np.mean(data)
  s = np.std(data)

  # identify outliers
  cut_off = s * 3.5 # pick any number of standard deviations (usually >= 2.0)
  lower, upper = m - cut_off, m + cut_off
  # identify outliers
  outliers = [x for x in data if x < lower or x > upper]
  print("{:d} outliers: min {:.2f} max {:.2f}".format(len(outliers), np.min(outliers), np.max(outliers)))
  
  # remove outliers
  # outliers_removed = [x for x in data if x >= lower and x <= upper]

find_fare_outliers(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(40)

In [0]:
def clean_sid(df):
   # need to implement this function
   # using the FunctionTransformer
   pass
   
def demo_custom_cleaning(df):
  try:
    # this will not work
    df['sid'] = df['sid'] - 1912 
  except Exception as e:
    print('invalid math')

  # clean it so we can do math on it
  df_c = clean_sid(df)

  df_c['sid'] = 1912 - df_c['sid']
  print(df_c.head())

demo_custom_cleaning(df.copy())

In [0]:
# run to read the next section
ide.reader.view_section(42)

# Lesson Assignment

In [0]:
# run to read the next section
ide.reader.view_section(43)

In [0]:
import numpy as np
import pandas as pd
import LessonUtil as Util

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

In [0]:
# run to read the next section
ide.reader.view_section(45)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(47)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(49)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(51)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(53)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(55)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(57)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(59)

In [0]:
# type&run the above example/exercise in this cell

In [0]:
# run to read the next section
ide.reader.view_section(61)

In [0]:
# type&run the above example/exercise in this cell

# Test and Submit

In [0]:
# run to read the next section
ide.reader.view_section(63)

In [0]:
# print(ide.tester.test_notebook()) 
# print(ide.tester.test_notebook(verbose=True)) 

# once you are ready -- run this 
# ide.tester.download_solution()